## ScenarioJudger

 - Reads a file from S3 containing json compliance scenarios of the format:
```json
{
  "scenarios": [
    {
      "scenario-id": "scenario-id-1",
      "scenario-detail": "A new employee, Sarah Johnson, joins the IT department...",
      "is-compliant": false
    },
    {
      "scenario-id": "scenario-id-2", 
      "scenario-detail": "TechCorp implements a comprehensive incident response procedure...",
      "is-compliant": true
    }
  ]
}
```
 - Evaluates the veracity each scenario-detail based on RAGed NIST-based policies in Bedrock knowledgebase, comparing its determination against "is-compliant" in the json.
 - When its determination differs, generates json records:
```json
{
  "scenarios": [
    {
      "scenario-id": "scenario-id-1",
      "scenario-detail": "A new employee, Sarah Johnson, joins the IT department...",
      "is-compliant": false,
      "judged-compliant": true,
      "judged-compliant-reason": "Considered the rules AC...  and scenario is not in violation..."
      "llm-judge": "us.anthropic.claude-sonnet-4-20250514-v1:0",
      "judged-dtm":  
    },
    {
      "scenario-id": "scenario-id-2", 
      "scenario-detail": "TechCorp implements a comprehensive incident response procedure...",
      "is-compliant": true,
      "judged-compliant": false,
      "judged-compliant-reason": "Scenario violates access control policy...",
      "llm-judge": "us.anthropic.claude-sonnet-4-20250514-v1:0",
      "judged-dtm":   
    }
  ]
}
```
 - Stores json records back to S3


In [ ]:
# Import required libraries
import boto3  # AWS SDK for Python
import json   # JSON handling
import time   # For rate limiting between API calls
from typing import List, Dict  # Type hints

# ============================================================================
# CONFIGURATION SECTION - Update these values
# ============================================================================
# S3 Configuration
INPUT_BUCKET = '183023889407-us-east-1-compliance-rule-generator'
INPUT_PREFIX = 'scenarios/'  # Folder path in S3 where PDFs are stored
OUTPUT_BUCKET = '183023889407-us-east-1-compliance-rule-generator'
OUTPUT_PREFIX = 'scenarios-judged/'  # Folder path for results
# AWS Region
AWS_REGION = 'us-east-1'
# AWS Bedrock Knowledge Base containing NIST policies
KNOWLEDGE_BASE_ID = 'T8EW10IU3Z'

MAX_TOKENS = 4096
TEMPERATURE = 0.7

# Available Bedrock model ARNs with performance notes
MODELS = {
    'premium': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-opus-4-5-20251101-v1:0', # not available
    'good': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0', # times out
    'balanced': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-sonnet-4-20250514-v1:0',  # recommended
    'fast_cheap': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-haiku-4-5-20251001-v1:0',
    'aws_native_premier': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-premier-v1:0',
    'aws_native_pro': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-pro-v1:0'
}
MODEL_ARN = MODELS['good']  # Default model selection

# JSON tool configuration for Bedrock Converse API
# Forces the model to return structured JSON with specific schema
TOOL_CONFIG = {
    "tools": [{
        "toolSpec": {
            "name": "judged_scenario_json",
            "description": "Return judged compliance scenarios as JSON",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "scenarios": {  # Array of scenario objects
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "judged-compliant": {"type": "boolean"},
                                    "judged-compliant-reason": {"type": "boolean"}
                                },
                                "required": ["judged-compliant", "judged-compliant-reason"]
                            }
                        }
                    },
                    "required": ["scenarios"]
                }
            }
        }
    }],
    "toolChoice": {"tool": {"name": "judged_scenario_json"}}  # Force use of the JSON tool
}

# Initialize AWS Bedrock clients
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')  # For knowledge base retrieval
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')  # For model inference